In [1]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

#inport custom functions 
#sys.path.append('../..')
#import functions

In [2]:
sr = 22050     # sample rate. based on Nyquist frequency, we only care about frequencies up to 10kHz therefor the sample rate will only perserve those frequencies 
sample_rate = sr
n_fft = 2048
hop_length = 512
duration = 30 # length of song to be used (in seconds) 
n_mels=128
n_mfcc=40

In [3]:
audio_dataset_path = '/Users/stephen/Flatiron/Phase4/MIR/Data/genres_original'
metadata= pd.read_csv('/Users/stephen/Flatiron/Phase4/MIR/Data/features_30_sec.csv')
metadata.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [ ]:
def recurrence(file_name):
    y, sr = librosa.load(file_name, res_type='kaiser_fast') 
    hop_length = 1024
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length)
    # time-delay embedding to get a cleaner recurrance matrix
    chroma_stack = librosa.feature.stack_memory(chroma, n_steps=10, delay=3)
    R = librosa.segment.recurrence_matrix(chroma_stack)
    return R

def rec_aff(file_name):
    y, sr = librosa.load(file_name, res_type='kaiser_fast') 
    hop_length = 1024
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length)
    # time-delay embedding to get a cleaner recurrance matrix
    chroma_stack = librosa.feature.stack_memory(chroma, n_steps=10, delay=3)
    R_aff = librosa.segment.recurrence_matrix(chroma_stack, metric='cosine', mode='affinity')
    return R_aff

In [ ]:
recurrence_features=[]
recurrence_affinity_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    try:
        final_class_labels=row["label"]
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row["filename"]))    
        data=recurrence(file_name)
        recurrence_features.append([data,final_class_labels])
        data2=rec_aff(file_name)
        recurrence_affinity_features.append([data,final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

In [ ]:
recurrence_features_df=pd.DataFrame(recurrence_features,columns=['recurrence','class'])
recurrence_features_df.head()

In [ ]:
recurrence_features_df['recurrence_affinity_features'] = recurrence_affinity_features
recurrence_features_df.head()